First, importing all packages which will be necessary through the process.

In [1]:
import pandas as pd
import numpy as np
!pip install geocoder
import geocoder
import geopy
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests 
from geopy.geocoders import Nominatim

Solving environment: done

# All requested packages already installed.



In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.cluster import KMeans 
from sklearn import metrics 
from scipy.spatial.distance import cdist 
import numpy as np 
import matplotlib.pyplot as plt  
import pandas as pd
from sklearn import preprocessing

import json

Using the read_html pandas feature, I have read the HTML tables on the Wikipedia page and then took the first one, Neighborhood name, Denonym, Population in 2010, Population per km², sub-neighborhoods and coordinayes.

In [3]:
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami", header=0)
tables = tables[0]
tables.head()

,Neighborhood,Demonym,Population2010,Population/Km²,Sub-neighborhoods,Coordinates
0,Allapattah,NaN,54289,4401,NaN,25.815-80.224
1,Arts & Entertainment District,NaN,11033,7948,NaN,25.799-80.190
2,Brickell,Brickellite,31759,14541,West Brickell,25.758-80.193
3,Buena Vista,NaN,9058,3540,Buena Vista East Historic District and Design ...,25.813-80.192
4,Coconut Grove,Grovite,20076,3091,"Center Grove, Northeast Coconut Grove, Southwe...",25.712-80.257


Doing the proper adjusts to the table, by dropping Rows with Coordinates or Population not assigned and dropping Columns which won't be important during the process.

In [4]:
tables = tables[tables['Coordinates'].notna()]
tables = tables[tables['Population/Km²'].notna()]
tables.drop(['Demonym','Sub-neighborhoods'],1, inplace=True)
tables.head()

,Neighborhood,Population2010,Population/Km²,Coordinates
0,Allapattah,54289,4401,25.815-80.224
1,Arts & Entertainment District,11033,7948,25.799-80.190
2,Brickell,31759,14541,25.758-80.193
3,Buena Vista,9058,3540,25.813-80.192
4,Coconut Grove,20076,3091,25.712-80.257


Finally, adjusting the latitude and longitude columns to be split and longitude to be negative, as well as one specific Population data which hasn't a number format.

In [5]:
tables[['Lat','Long']] = tables.Coordinates.str.split("-",expand=True)
tables.drop('Coordinates',1, inplace=True)
tables.head()

,Neighborhood,Population2010,Population/Km²,Lat,Long
0,Allapattah,54289,4401,25.815,80.224
1,Arts & Entertainment District,11033,7948,25.799,80.190
2,Brickell,31759,14541,25.758,80.193
3,Buena Vista,9058,3540,25.813,80.192
4,Coconut Grove,20076,3091,25.712,80.257


In [6]:
tables['Lat'] = pd.to_numeric(tables['Lat'])
tables['Long'] = pd.to_numeric(tables['Long'])
tables.at[7,'Population2010']= 71000
indexNames = tables[ tables['Population/Km²'] == '-' ].index
tables.drop(indexNames , inplace=True)
tables['Population2010'] = pd.to_numeric(tables['Population2010'])
tables['Population/Km²'] = pd.to_numeric(tables['Population/Km²'])
tables['Long'] = tables.Long * (-1)
tables = tables.reset_index(drop=True)
tables.head()

,Neighborhood,Population2010,Population/Km²,Lat,Long
0,Allapattah,54289,4401,25.815,-80.224
1,Arts & Entertainment District,11033,7948,25.799,-80.190
2,Brickell,31759,14541,25.758,-80.193
3,Buena Vista,9058,3540,25.813,-80.192
4,Coconut Grove,20076,3091,25.712,-80.257


In [7]:
address = 'Miami'
geolocator = Nominatim(user_agent="exploring_Miami")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Miami are {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Miami are 25.7742658, -80.1936589.


Now with the data well formatted, next step is to grab data from Foursquare and add it to our Dataframe, in order to be possible to analyze it with K-Means.

In [8]:
CLIENT_ID = 'EX3NXMNX5TLNEGG5KTIXLABHKWAIQZP4WBMG4CPGTSCD0DQ4' # your Foursquare ID
CLIENT_SECRET = 'BFVHVLQ352ZNOHCBJUMSCZ2SNGZOV34TQX5TWQ2D1SHVX5NT' # your Foursquare Secret
VERSION = '20200415' # Foursquare API version
LIMIT = 3000 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
miamivenues = getNearbyVenues(names=tables['Neighborhood'],
                                   latitudes=tables['Lat'],
                                   longitudes=tables['Long']
                                  )



Allapattah
Arts & Entertainment District
Brickell
Buena Vista
Coconut Grove
Coral Way
Design District
Downtown
Edgewater
Flagami
Grapeland Heights
Liberty City
Little Haiti
Little Havana
Lummus Park
Overtown
Park West
The Roads
Upper Eastside
West Flagler
Wynwood


In [11]:
print(miamivenues.shape)
miamivenues

(444, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Allapattah,25.815,-80.224,Three Fingers Liquor & Lounge,25.815408,-80.224361,Lounge
1,Allapattah,25.815,-80.224,Ross,25.815820,-80.221753,Department Store
2,Allapattah,25.815,-80.224,Showtime Boxing Gym,25.812364,-80.224504,Boxing Gym
3,Arts & Entertainment District,25.799,-80.190,Bunnie Cakes,25.799544,-80.190953,Cupcake Shop
4,Arts & Entertainment District,25.799,-80.190,Bunbury Miami,25.798284,-80.191118,Wine Shop
5,Arts & Entertainment District,25.799,-80.190,Yodi's Threading Spa,25.800490,-80.189093,Spa
6,Arts & Entertainment District,25.799,-80.190,Plant Food + Wine Miami,25.800452,-80.192805,Restaurant
7,Arts & Entertainment District,25.799,-80.190,TRIO - Mind Body Spirit Studio,25.799573,-80.190972,Yoga Studio
8,Arts & Entertainment District,25.799,-80.190,The Daily Creative Food Co. - Miami,25.795696,-80.188683,Sandwich Place
9,Arts & Entertainment District,25.799,-80.190,LegacyFit,25.800222,-80.193052,Gym


In [12]:
miamicategories = pd.DataFrame() 
miamicategories['miamicategories'] = miamivenues.groupby('Neighborhood').count()['Venue Category']
miamicategories.reset_index(inplace=True)
miamicategories

,Neighborhood,miamicategories
0,Allapattah,3
1,Arts & Entertainment District,19
2,Brickell,58
3,Buena Vista,26
4,Coconut Grove,4
5,Coral Way,10
6,Design District,26
7,Downtown,50
8,Edgewater,48
9,Flagami,8


In [13]:
miamitop = pd.get_dummies(miamivenues[['Venue Category']], prefix="", prefix_sep="")
miamitop['ZNeighborhood'] = miamivenues['Neighborhood'] 
rightcolumns = [miamitop.columns[-1]] + list(miamitop.columns[:-1])
miamitop = miamitop[rightcolumns]
miamitop = miamitop.rename(columns={'ZNeighborhood':'Correct Neighborhood'})

miamitop.head()

,Correct Neighborhood,Accessories Store,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Stadium,Big Box Store,Bistro,Boat or Ferry,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,Home Service,Hotel,Hotel Pool,Ice Cream Shop,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Lawyer,Leather Goods Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Motel,Moving Target,Museum,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Print Shop,Pub,Public Art,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio
0,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Allapattah,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Arts & Entertainment District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Arts & Entertainment District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [14]:
miamigroup = miamitop.groupby('Correct Neighborhood').mean().reset_index()
miamigroup

,Correct Neighborhood,Accessories Store,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Stadium,Big Box Store,Bistro,Boat or Ferry,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,Home Service,Hotel,Hotel Pool,Ice Cream Shop,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Lawyer,Leather Goods Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Motel,Moving Target,Museum,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Print Shop,Pub,Public Art,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio
0,Allapattah,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.333333,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.333333,0.0000,0.0000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.000000,0.00,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
1,Arts & Entertainment District,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.052632,0.000000,0.000000,0.000000,0.00,0.000000,0.052632,0.000000,0.000000,0.0000,0.0000,0.000000,0.0,0.000000,0.000000,0.0000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.052632,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.105263,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.0000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000,0.052632,0.00,0

In [15]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [16]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = miamigroup['Correct Neighborhood']

for ind in np.arange(miamigroup.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(miamigroup.iloc[ind, :], num_top_venues)


miamigroupcluster = miamigroup.drop('Correct Neighborhood', 1)
miamigroupcluster.head()

,Accessories Store,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Stadium,Big Box Store,Bistro,Boat or Ferry,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,Home Service,Hotel,Hotel Pool,Ice Cream Shop,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Lawyer,Leather Goods Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Motel,Moving Target,Museum,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Print Shop,Pub,Public Art,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio
0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.333333,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000
1,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.052632,0.0,0.0,0.000000,0.0,0.0,0.052632,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.052632,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105263,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.052632,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.052632,0.0,0.000000,0.000000,0.000000,0.0,0.052632,0.052632,0.0,0.0,0.0,0.0,0.000000,0.0,0.052632,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.157895,0.000000,0.052632,0.052632,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.052632,0.000000,0.0,0.000000,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.000000,0.000000,0.052632,0.0,0.0,0.052632
2,0.0,0.034483,0.0,0.000000,0.034483,0.000000,0.000000,0.000000,0.017241,0.0,0.0,0.034483,0.034483,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.017241,0.0,0.051724,0.0,0.0,0.000000,0.0,0

Adding population, density and number of venues data into the new table, the three normalized.

In [17]:
x = tables[['Population2010']].values.astype(float)
y = tables[['Population/Km²']].values.astype(float)
z = miamicategories[['miamicategories']].values.astype(float)

min_max_scaler = preprocessing.MinMaxScaler()
Pop_scaled = min_max_scaler.fit_transform(x)
Dens_scaled = min_max_scaler.fit_transform(y)
Number_of_categories = min_max_scaler.fit_transform(z)
miamigroupcluster['Pop_scaled']=Pop_scaled
miamigroupcluster['Dens_scaled']=Dens_scaled
miamigroupcluster['Number of categories']=Number_of_categories
miamigroupcluster.head()



,Accessories Store,American Restaurant,Aquarium,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Stadium,Big Box Store,Bistro,Boat or Ferry,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Business Service,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cupcake Shop,Dance Studio,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Empanada Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Fish Market,Flea Market,Food,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gastropub,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,Home Service,Hotel,Hotel Pool,Ice Cream Shop,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Latin American Restaurant,Lawyer,Leather Goods Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Mobile Phone Shop,Motel,Moving Target,Museum,Neighborhood,New American Restaurant,Nightclub,Optical Shop,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Print Shop,Pub,Public Art,Record Shop,Residential Building (Apartment / Condo),Resort,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Seafood Restaurant,Shoe Store,Shopping Mall,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Steakhouse,Sushi Restaurant,Taco Place,Tapas Restaurant,Tennis Court,Theater,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Wine Bar,Wine Shop,Winery,Wings Joint,Yoga Studio,Pop_scaled,Dens_scaled,Number of categories
0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.333333,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.333333,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.700913,0.156967,0.012821
1,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.052632,0.0,0.0,0.000000,0.0,0.0,0.052632,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.052632,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.105263,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.052632,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.052632,0.0,0.000000,0.000000,0.000000,0.0,0.052632,0.052632,0.0,0.0,0.0,0.0,0.000000,0.0,0.052632,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.157895,0.000000,0.052632,0.052632,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.052632,0.000000,0.0,0.000000,0.0,0.0,0.052632,0.0,0.0,0.0,0.0,0.000000,0.000000,0.052632,0.0,0.0,0.052632,0.109467,0.451862,0.217949
2,0.0,0.034483,0.0,0.000000,0.034483,0.000000,0.000000,0.000000,0.017241,0.0,0.0,0.034483,0.03448

Since we have only 21 neighborhoods, the data will be divided among 5 clusters.

In [18]:
# set number of clusters
kclusters = 5



# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(miamigroupcluster)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 3, 4, 0, 1, 1, 0, 4, 3, 1], dtype=int32)

In [19]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

miamimerge = tables

miamimerge = miamimerge.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')



In [20]:
miamimerge= miamimerge.dropna()
miamimerge['Cluster Labels'] = miamimerge['Cluster Labels'].apply(np.int64)
miamimerge

,Neighborhood,Population2010,Population/Km²,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allapattah,54289,4401,25.815,-80.224,1,Boxing Gym,Department Store,Lounge,Event Space,Food,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Yoga Studio
1,Arts & Entertainment District,11033,7948,25.799,-80.190,3,Restaurant,Gym,Yoga Studio,Spa,Ice Cream Shop,Food Truck,Moving Target,Cupcake Shop,Paper / Office Supplies Store,Park
2,Brickell,31759,14541,25.758,-80.193,4,Hotel,Italian Restaurant,Japanese Restaurant,Café,Bank,Juice Bar,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Bar
3,Buena Vista,9058,3540,25.813,-80.192,0,Jewelry Store,Café,Boutique,Italian Restaurant,Coffee Shop,Furniture / Home Store,Shopping Mall,Fast Food Restaurant,Big Box Store,Mediterranean Restaurant
4,Coconut Grove,20076,3091,25.712,-80.257,1,American Restaurant,Boat or Ferry,Fried Chicken Joint,Park,Yoga Studio,Empanada Restaurant,Fish Market,Fast Food Restaurant,Farmers Market,Event Space
5,Coral Way,35062,4496,25.750,-80.283,1,Liquor Store,Pharmacy,Dive Bar,Seafood Restaurant,Burger Joint,Café,Golf Course,Mobile Phone Shop,Historic Site,Discount Store
6,Design District,3573,3623,25.813,-80.193,0,Jewelry Store,Café,Boutique,Italian Restaurant,Coffee Shop,Furniture / Home Store,Shopping Mall,Fast Food Restaurant,Big Box Store,Mediterranean Restaurant
7,Downtown,71000,10613,25.774,-80.193,4,Hotel,Peruvian Restaurant,Café,Italian Restaurant,Brazilian Restaurant,Cosmetics Shop,Gym,Pharmacy,Coffee Shop,Lounge
8,Edgewater,15005,6675,25.802,-80.190,3,Restaurant,Gym,Sandwich Place,Art Gallery,Breakfast Spot,Pizza Place,Peruvian Restaurant,Cuban Restaurant,Coffee Shop,Ice Cream Shop
9,Flagami,50834,5665,25.762,-80.316,1,Seafood Restaurant,Bakery,Pet Store,Department Store,Fast Food Restaurant,Spanish Restaurant,Latin American Restaurant,Fish Market,Farmers Market,Event Space


Now, getting Mandolin's location and calculating its distance from all the neighborhoods centerpoints. Then, this data will be added to the dataframe.

In [21]:
mandolin = '4312 NE 2nd Ave, Miami, FL 33137'
geolocator = Nominatim(user_agent="mandolin")
locationm = geolocator.geocode(mandolin)
latitudem = locationm.latitude
longitudem = locationm.longitude
print('The geograpical coordinate of Mandolin are {}, {}.'.format(latitudem, longitudem))

The geograpical coordinate of Mandolin are 25.81577787755102, -80.19156669387755.


In [22]:
def dist (names, lat, lon):
    from math import sin, cos, sqrt, atan2, radians

    # approximate radius of earth in km
    R = 6373.0
    dist_list=[]
    for name, lat, lon in zip(names, lat, lon):
        print(name)
        lat1 = radians(latitudem)
        lon1 = radians(longitudem)
        lat2 = radians(lat)
        lon2 = radians(lon)

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
        dist_list.append([(
            name, 
            lat, 
            lon,
            distance)])
    distancemiami = pd.DataFrame([item for dist_list in dist_list for item in dist_list])
    distancemiami.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Distance']
    return (distancemiami)

In [23]:
miamidistance = dist(names=tables['Neighborhood'],
                                   lat=tables['Lat'],
                                   lon=tables['Long']
                                  )

Allapattah
Arts & Entertainment District
Brickell
Buena Vista
Coconut Grove
Coral Way
Design District
Downtown
Edgewater
Flagami
Grapeland Heights
Liberty City
Little Haiti
Little Havana
Lummus Park
Overtown
Park West
The Roads
Upper Eastside
West Flagler
Wynwood


In [24]:
miamidistance
miamimerge['Distance'] = miamidistance['Distance']
miamimerge['Number of Venues'] = miamicategories['miamicategories']
miamimerge

,Neighborhood,Population2010,Population/Km²,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Distance,Number of Venues
0,Allapattah,54289,4401,25.815,-80.224,1,Boxing Gym,Department Store,Lounge,Event Space,Food,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Yoga Studio,3.248677,3
1,Arts & Entertainment District,11033,7948,25.799,-80.190,3,Restaurant,Gym,Yoga Studio,Spa,Ice Cream Shop,Food Truck,Moving Target,Cupcake Shop,Paper / Office Supplies Store,Park,1.872783,19
2,Brickell,31759,14541,25.758,-80.193,4,Hotel,Italian Restaurant,Japanese Restaurant,Café,Bank,Juice Bar,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Bar,6.428227,58
3,Buena Vista,9058,3540,25.813,-80.192,0,Jewelry Store,Café,Boutique,Italian Restaurant,Coffee Shop,Furniture / Home Store,Shopping Mall,Fast Food Restaurant,Big Box Store,Mediterranean Restaurant,0.312014,26
4,Coconut Grove,20076,3091,25.712,-80.257,1,American Restaurant,Boat or Ferry,Fried Chicken Joint,Park,Yoga Studio,Empanada Restaurant,Fish Market,Fast Food Restaurant,Farmers Market,Event Space,13.274360,4
5,Coral Way,35062,4496,25.750,-80.283,1,Liquor Store,Pharmacy,Dive Bar,Seafood Restaurant,Burger Joint,Café,Golf Course,Mobile Phone Shop,Historic Site,Discount Store,11.721493,10
6,Design District,3573,3623,25.813,-80.193,0,Jewelry Store,Café,Boutique,Italian Restaurant,Coffee Shop,Furniture / Home Store,Shopping Mall,Fast Food Restaurant,Big Box Store,Mediterranean Restaurant,0.340687,26
7,Downtown,71000,10613,25.774,-80.193,4,Hotel,Peruvian Restaurant,Café,Italian Restaurant,Brazilian Restaurant,Cosmetics Shop,Gym,Pharmacy,Coffee Shop,Lounge,4.649163,50
8,Edgewater,15005,6675,25.802,-80.190,3,Restaurant,Gym,Sandwich Place,Art Gallery,Breakfast Spot,Pizza Place,Peruvian Restaurant,Cuban Restaurant,Coffee Shop,Ice Cream Shop,1.540520,48
9,Flagami,50834,5665,25.762,-80.316,1,Seafood Restaurant,Bakery,Pet Store,Department Store,Fast Food Restaurant,Spanish Restaurant,Latin American Restaurant,Fish Market,Farmers Market,Event Space,13.823434,8


In [25]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(miamimerge['Lat'], miamimerge['Long'], miamimerge['Neighborhood'], miamimerge['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
folium.Marker(location=[latitudem,longitudem], icon=folium.Icon(color='lightgray', icon='home', prefix='fa')).add_to(map_clusters)
map_clusters

Since our neighboorhood of interest - Designer district - is in Cluster 0, we want to filter only neighborhoods in this Cluster and order it by distance. In this way, it will be easy to check which is the neighborhood with more distance from the actual one.

In [26]:
interestneighborhoods =  miamimerge[miamimerge['Cluster Labels']==0]
interestneighborhoods = interestneighborhoods.sort_values('Distance')
interestneighborhoods

,Neighborhood,Population2010,Population/Km²,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Distance,Number of Venues
3,Buena Vista,9058,3540,25.813,-80.192,0,Jewelry Store,Café,Boutique,Italian Restaurant,Coffee Shop,Furniture / Home Store,Shopping Mall,Fast Food Restaurant,Big Box Store,Mediterranean Restaurant,0.312014,26
6,Design District,3573,3623,25.813,-80.193,0,Jewelry Store,Café,Boutique,Italian Restaurant,Coffee Shop,Furniture / Home Store,Shopping Mall,Fast Food Restaurant,Big Box Store,Mediterranean Restaurant,0.340687,26
18,Upper Eastside,12525,2513,25.830,-80.183,0,Gas Station,Pizza Place,Motel,Intersection,Tennis Court,Italian Restaurant,Furniture / Home Store,Sushi Restaurant,Yoga Studio,Farmers Market,1.799492,8
15,Overtown,6736,3405,25.787,-80.201,0,Wings Joint,Southern / Soul Food Restaurant,Baseball Stadium,Farmers Market,Athletics & Sports,Event Space,Food,Flea Market,Fish Market,Fast Food Restaurant,3.337442,5
16,Park West,4655,3635,25.785,-80.193,0,Nightclub,Café,Bar,Lounge,Theater,Hotel,Gastropub,Japanese Restaurant,Flea Market,Event Space,3.426426,28
14,Lummus Park,3027,3680,25.777,-80.201,0,Seafood Restaurant,Hotel,American Restaurant,Fish Market,BBQ Joint,Historic Site,Restaurant,Spanish Restaurant,Soccer Field,Golf Course,4.415501,14
17,The Roads,7327,4899,25.756,-80.207,0,Lawyer,Health & Beauty Service,Moving Target,Business Service,Frozen Yogurt Shop,Gas Station,Italian Restaurant,Latin American Restaurant,Farmers Market,Fast Food Restaurant,6.826385,9
10,Grapeland Heights,14004,4130,25.792,-80.258,0,Hotel,Gym / Fitness Center,Restaurant,Bar,Gym,Golf Course,Hotel Pool,Dessert Shop,Diner,Discount Store,7.159026,7


In [27]:
miamimerge[miamimerge['Cluster Labels']==1]

,Neighborhood,Population2010,Population/Km²,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Distance,Number of Venues
0,Allapattah,54289,4401,25.815,-80.224,1,Boxing Gym,Department Store,Lounge,Event Space,Food,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Yoga Studio,3.248677,3
4,Coconut Grove,20076,3091,25.712,-80.257,1,American Restaurant,Boat or Ferry,Fried Chicken Joint,Park,Yoga Studio,Empanada Restaurant,Fish Market,Fast Food Restaurant,Farmers Market,Event Space,13.274360,4
5,Coral Way,35062,4496,25.750,-80.283,1,Liquor Store,Pharmacy,Dive Bar,Seafood Restaurant,Burger Joint,Café,Golf Course,Mobile Phone Shop,Historic Site,Discount Store,11.721493,10
9,Flagami,50834,5665,25.762,-80.316,1,Seafood Restaurant,Bakery,Pet Store,Department Store,Fast Food Restaurant,Spanish Restaurant,Latin American Restaurant,Fish Market,Farmers Market,Event Space,13.823434,8
11,Liberty City,19725,3733,25.832,-80.225,1,Southern / Soul Food Restaurant,Grocery Store,Donut Shop,Flea Market,Fish Market,Fast Food Restaurant,Farmers Market,Event Space,Empanada Restaurant,Dog Run,3.802759,2
12,Little Haiti,29760,3840,25.824,-80.191,1,Yoga Studio,Shopping Mall,Gas Station,Ice Cream Shop,Italian Restaurant,Fast Food Restaurant,Liquor Store,Donut Shop,Dance Studio,Coffee Shop,0.916304,23
19,West Flagler,31407,4428,25.775,-80.243,1,Cuban Restaurant,Bakery,Pharmacy,Concert Hall,Comfort Food Restaurant,Gas Station,Latin American Restaurant,Fast Food Restaurant,Farmers Market,Event Space,6.863234,7


In [28]:
miamimerge[miamimerge['Cluster Labels']==2]

,Neighborhood,Population2010,Population/Km²,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Distance,Number of Venues
13,Little Havana,76163,8423,25.773,-80.215,2,Latin American Restaurant,Shoe Store,Pharmacy,Grocery Store,Discount Store,Mexican Restaurant,Restaurant,Fast Food Restaurant,Farmers Market,Event Space,5.305431,9


In [29]:
miamimerge[miamimerge['Cluster Labels']==3]

,Neighborhood,Population2010,Population/Km²,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Distance,Number of Venues
1,Arts & Entertainment District,11033,7948,25.799,-80.190,3,Restaurant,Gym,Yoga Studio,Spa,Ice Cream Shop,Food Truck,Moving Target,Cupcake Shop,Paper / Office Supplies Store,Park,1.872783,19
8,Edgewater,15005,6675,25.802,-80.190,3,Restaurant,Gym,Sandwich Place,Art Gallery,Breakfast Spot,Pizza Place,Peruvian Restaurant,Cuban Restaurant,Coffee Shop,Ice Cream Shop,1.540520,48
20,Wynwood,7277,2983,25.804,-80.199,3,Art Gallery,Bar,Café,Food Truck,Theater,Peruvian Restaurant,Bakery,Caribbean Restaurant,Public Art,Coffee Shop,1.506737,80


In [30]:
miamimerge[miamimerge['Cluster Labels']==4]

,Neighborhood,Population2010,Population/Km²,Lat,Long,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Distance,Number of Venues
2,Brickell,31759,14541,25.758,-80.193,4,Hotel,Italian Restaurant,Japanese Restaurant,Café,Bank,Juice Bar,Salon / Barbershop,Sandwich Place,Seafood Restaurant,Bar,6.428227,58
7,Downtown,71000,10613,25.774,-80.193,4,Hotel,Peruvian Restaurant,Café,Italian Restaurant,Brazilian Restaurant,Cosmetics Shop,Gym,Pharmacy,Coffee Shop,Lounge,4.649163,50
